# Guidelines cards example

This is an example to help you get started with Media Laboratory visualizations,
which also makes use of the [Guideline cards](https://news-teller.github.io/media-laboratory-web/#/guidelines).  
The step-by-step guide is available [here](https://news-teller.github.io/media-laboratory-web/#/developers).

In [ ]:
import os
import dataviz

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [ ]:
# Create main plot
df = pd.DataFrame({
    "Source": [
        "Agriculture", "Déchets", "Energie (centrales électriques)", "Industrie (combustion)",
        "Industrie (procédés)", "Transports", "Résidentiel", "Tertiaire"
    ],
    "Amount": [9, 1, 11, 26, 16, 19, 14, 4],
})

fig = px.pie(df, values="Amount", names="Source", color_discrete_sequence=px.colors.sequential.RdBu)


# Create Dash application
UID = '9d61311fd159'

app = JupyterDash(
    title='Guidelines cards example',
    assets_folder=os.getenv("ASSETS_FOLDER", "") + "/" + UID,
    external_scripts=[
        'https://unpkg.com/@popperjs/core@2',
        'https://news-teller.github.io/media-laboratory-web/popovers.js',
    ]
)

# Construct layout
def get_popover_element(text: str, card_id: str) -> html:
    """Construct a <i> tag to trigger a popover. """
    data_attrs = {
        "data-toggle": "popovercard",
        "data-card-id": card_id,
    }
    
    return html.I(text, **data_attrs)

header = html.Div(children=[
    html.H1("Guidelines cards example"),
    html.H2(children=[
        "Emissions de ",
        get_popover_element("gaz à effet de serre", card_id="30")
    ]),  
])

footer = html.Div(children=[
    get_popover_element("Le CO2", card_id="54"),
    """
    qui représente 86 % des émissions totales de gaz à effet de serre,
    est principalement émis lors des processus de combustion:
    industrie, transports, chauffage résidentiel, industries de production d’électricité et tertiaire. 
    """
])

# we'll need these later to define a clientside callback
empty_divs = html.Div(children=[
    html.Div(id='blank-input'),
    html.Div(id='blank-output')
], className="clientside-io")


app.layout = html.Div(children=[
    header,
    dcc.Graph(figure=fig),
    footer,
    empty_divs,
], className="wrapper")


# define a callback to load popovers card when the page is fully loaded
app.clientside_callback(
    """
    function() {
        const showCards = window['reloadPopoversjsElements'];
        if (showCards) showCards();
    }
    """,
    Output('blank-output', 'children'),
    Input('blank-input', 'children'),
)

In [ ]:
JupyterDash.infer_jupyter_proxy_config()
app.run_server(mode="external", port='8051')

In [ ]:
# the `DataViz` helps manage (save,list,retrieve) the visualizations
dz = dataviz.DataViz()

In [ ]:
# save the viz created above
dz.store(
    uid=UID,
    title=app.title,
    dash_app=app,
    tags=['notebook', 'example'] # tags are optional
)